# Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

from google.colab import files # to download files

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Dataset


In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

In [3]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="hXpSwYyT2s5LPrxuiYox")
project = rf.workspace("m-zrubec60-gmail-com").project("billboards")
dataset = project.version(3).download("yolov5")

# Train

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
!python train.py --img 640 --batch 4 --epochs 40 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

# Training results

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!zip -r training_results.zip ./runs/train/exp/

In [ ]:
files.download('./training_results.zip')

# Validate

In [ ]:
!python val.py --weights ./best.pt --data {dataset.location}/data.yaml

In [ ]:
!zip -r validate_img_results.zip ./runs/val/exp/

In [7]:
files.download('./test_img_results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Detect

Images

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.35 --source {dataset.location}/test/images

In [ ]:
!zip -r test_img_results.zip ./runs/detect/exp3/

In [ ]:
files.download('./detect_img_results.zip')

Video

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.35 --source https://www.youtube.com/watch?v=BP5p8C2lzok&ab_channel=LiviuNica

In [ ]:
files.download('/content/yolov5/runs/detect/exp2/*.mp4')

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

package = 10  # the first N results
exp = 2

for imageName in glob.glob(f'/content/yolov5/runs/detect/exp{exp}/*.jpg')[:package]: #assuming JPG
    display(Image(filename=imageName))
    print("\n")